# Segmentation of Brain Tissues

## The Dataset

### Investigating the Dataset

### Parameters

In [22]:
import os
import glob
import warnings
from torch.utils.data import DataLoader
from Q1_dataprep import IBSRPatchDataset
BASE_DIR = r"C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped"

volume_files = []
segmentation_files = []

# Loop over all subject folders inside IBSR_nifti_stripped
for subj_name in sorted(os.listdir(BASE_DIR)):
    subj_path = os.path.join(BASE_DIR, subj_name)
    if not os.path.isdir(subj_path):
        continue  # skip non-folder entries just in case

    # Look for volume and segmentation files inside each subject folder
    vol_candidates = glob.glob(os.path.join(subj_path, "*_ana.nii*"))
    seg_candidates = glob.glob(os.path.join(subj_path, "*_ana_brainmask.nii*"))

    if not vol_candidates or not seg_candidates:
        warnings.warn(f"Could not find volume or segmentation in {subj_path}")
        continue

    # Take the first match (there should usually be exactly one)
    vol_path = vol_candidates[0]
    seg_path = seg_candidates[0]

    volume_files.append(vol_path)
    segmentation_files.append(seg_path)

print("Found volumes:")
for v in volume_files:
    print("  ", v)

print("Found segmentations:")
for s in segmentation_files:
    print("  ", s)

# Create your dataset
dataset = IBSRPatchDataset(volume_files, segmentation_files)

# (Optional) Wrap with DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Quick sanity check: pull one batch
for batch_vol, batch_mask in dataloader:
    print("Batch volume shape:", batch_vol.shape)  # [B, 1, 128, 128]
    print("Batch mask shape:", batch_mask.shape)   # [B, 128, 128]
    break


Found volumes:
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_01\IBSR_01_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_02\IBSR_02_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_03\IBSR_03_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_04\IBSR_04_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_05\IBSR_05_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_06\IBSR_06_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_07\IBSR_07_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_08\IBSR_08_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_09\IBSR_09_ana.nii.gz
   C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_s

c:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\Q1_dataprep.py:91: UserWarning: Warning: Skipping file C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_01\IBSR_01_ana.nii.gz due to error: index 130 is out of bounds for axis 1 with size 128
  warnings.warn(f"Warning: Skipping file {vol_path} due to error: {e}")
c:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\Q1_dataprep.py:91: UserWarning: Warning: Skipping file C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_02\IBSR_02_ana.nii.gz due to error: index 130 is out of bounds for axis 1 with size 128
  warnings.warn(f"Warning: Skipping file {vol_path} due to error: {e}")
c:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\Q1_dataprep.py:91: UserWarning: Warning: Skipping file C:\Users\a\Desktop\7th semester\DL\NNDL_Assignment3\IBSR_nifti_stripped\IBSR_03\IBSR_03_ana.nii.gz due to error: index 130 is out of bounds for axis 1 with size 128
  warnings.warn(f"Warning: Skipping file

Dataset loaded. Total patches: 6336


RuntimeError: stack expects each tensor to be equal size, but got [128, 128] at entry 0 and [128, 0] at entry 1

Creating training dataset...
Loading and processing dataset (with rotation)...
Dataset loaded. Total patches: 6528
Creating testing dataset...
Loading and processing dataset (with rotation)...
Dataset loaded. Total patches: 3264

========== DATA OVERVIEW ==========
Number of training subjects: 12
Number of testing subjects : 6
Number of training patches : 6528
Number of testing patches  : 3264

Example input patch tensor shape: torch.Size([1, 128, 128])
Example mask patch tensor shape : torch.Size([128, 128])
Example input dtype: torch.float32
Example mask dtype : torch.int64

Input patch stats:
  min: 0.0
  max: 0.0
  mean: 0.0

Mask unique labels in example patch: [0]

Approx. patches per training subject: 544.0
Approx. patches per testing subject : 544.0



In [20]:
for i in range(20000):
    vol, mask = train_dataset[i]
    if vol.max() > 0:
        print("Found a non-empty patch at index:", i)
        print("Patch stats:", vol.min(), vol.max(), vol.mean())
        break



Found a non-empty patch at index: 64
Patch stats: tensor(0.) tensor(3.) tensor(0.0326)
